# Reporte de Calidad de Datos - Municipios PDET

Este notebook valida y resume la calidad de la colección `pdet_municipalities` en MongoDB.
Incluye exportación de estadísticas y generación de `pdet_region_stats.csv`.

In [ ]:
%pip install --quiet geopandas pandas pymongo shapely

import os
import pandas as pd
import geopandas as gpd
from pymongo import MongoClient
from shapely.geometry import shape

MONGO_URI = 'mongodb://localhost:27017'
DB_NAME = 'pdet_solar_analysis'

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
print('Conectado a MongoDB:', DB_NAME)

os.makedirs('results', exist_ok=True)


## Cargar colección `pdet_municipalities` desde MongoDB

In [ ]:
def load_pdet_from_mongo(client, db_name, coll_name='pdet_municipalities', geom_field='geometry'):
    coll = client[db_name][coll_name]
    docs = list(coll.find({}))
    if len(docs) == 0:
        print('Colección vacía o no encontrada')
        return gpd.GeoDataFrame()
    df = pd.DataFrame(docs)
    if geom_field in df.columns:
        df['geometry'] = df[geom_field].apply(lambda g: shape(g) if isinstance(g, dict) else g)
        gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
    else:
        gdf = gpd.GeoDataFrame(df)
    return gdf

pdet_gdf = load_pdet_from_mongo(client, DB_NAME)
print('Municipios PDET cargados:', len(pdet_gdf))

## Validaciones básicas

In [ ]:
# Chequeos de calidad
report = {}
report['total_docs'] = len(pdet_gdf)
report['null_values'] = pdet_gdf.isnull().sum().to_dict()
report['duplicate_divipola'] = int(pdet_gdf['divipola_code'].duplicated().sum()) if 'divipola_code' in pdet_gdf.columns else 'no_divipola'
try:
    report['invalid_geometries'] = int((~pdet_gdf.is_valid).sum())
except Exception:
    report['invalid_geometries'] = 'no_geometry'

print('===== RESUMEN DE CALIDAD =====')
for k,v in report.items():
    print(k, ':', v)

display(pdet_gdf.head(3))

## Estadísticas por región PDET (usando centroides de municipios si fuera necesario)
Agrupamos por `region_pdet` y exportamos el CSV resultante a `results/pdet_region_stats.csv`.

In [ ]:
import os
from shapely.geometry import Point

# Algunos shapefiles pueden necesitar centroides: si la geometría de municipio es polígono, se usa su propio centroid para ciertos análisis.
if 'geometry' in pdet_gdf.columns:
    # crear una columna centroid_muni por si es necesario
    pdet_gdf['centroid_muni'] = pdet_gdf.geometry.centroid

if 'region_pdet' in pdet_gdf.columns:
    region_stats = pdet_gdf.groupby('region_pdet').agg(
        municipios=('municipio', 'count')
    ).reset_index()
    # Si existe area_km2, añadir
    if 'area_km2' in pdet_gdf.columns:
        region_stats['area_total'] = pdet_gdf.groupby('region_pdet')['area_km2'].sum().values
        region_stats['area_promedio'] = pdet_gdf.groupby('region_pdet')['area_km2'].mean().values
    os.makedirs('results', exist_ok=True)
    path = os.path.join('results', 'pdet_region_stats.csv')
    region_stats.to_csv(path, index=False)
    print('✅ region_stats exportado a', os.path.abspath(path))
    display(region_stats)
else:
    print("La columna 'region_pdet' no existe en pdet_gdf")

## Guardado de reporte completo (JSON/CSV)
Exportamos también el diccionario `report` como archivo JSON para su inclusión en el entregable.

In [ ]:
import json
with open('results/data_quality_summary.json','w',encoding='utf-8') as f:
    json.dump(report, f, indent=2, ensure_ascii=False)
pd.DataFrame.from_dict(report, orient='index', columns=['value']).to_csv('results/data_quality_summary.csv')
print('Resúmenes exportados a results/')

## Conclusiones
- Este notebook valida la colección de municipios PDET y exporta estadísticas por región.
- El CSV `results/pdet_region_stats.csv` contiene el número de municipios por región y (si está disponible) estadísticas de área.
